# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fac6a1c9820>
├── 'a' --> tensor([[ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904]])
└── 'x' --> <FastTreeValue 0x7fac6a1c9f70>
    └── 'c' --> tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                        [ 0.2619, -0.3360, -0.7763, -0.8405],
                        [ 1.1931, -0.9129,  0.8167,  0.7423]])

In [4]:
t.a

tensor([[ 0.9432,  0.5049, -0.6372],
        [ 0.2895,  0.3341, -0.0904]])

In [5]:
%timeit t.a

58.3 ns ± 0.756 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fac6a1c9820>
├── 'a' --> tensor([[-0.0217,  0.9443, -0.4712],
│                   [ 1.1626,  1.4264, -0.9823]])
└── 'x' --> <FastTreeValue 0x7fac6a1c9f70>
    └── 'c' --> tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                        [ 0.2619, -0.3360, -0.7763, -0.8405],
                        [ 1.1931, -0.9129,  0.8167,  0.7423]])

In [7]:
%timeit t.a = new_value

65.3 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9432,  0.5049, -0.6372],
               [ 0.2895,  0.3341, -0.0904]]),
    x: Batch(
           c: tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9432,  0.5049, -0.6372],
        [ 0.2895,  0.3341, -0.0904]])

In [11]:
%timeit b.a

57.3 ns ± 0.832 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.3384,  0.8589, -0.3298],
               [ 0.9015,  0.2753,  0.6456]]),
    x: Batch(
           c: tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423]]),
       ),
)

In [13]:
%timeit b.a = new_value

460 ns ± 6.07 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

779 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

9.27 µs ± 82.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 3.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fabb0381a30>
├── 'a' --> tensor([[[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]],
│           
│                   [[ 0.9432,  0.5049, -0.6372],
│                    [ 0.2895,  0.3341, -0.0904]]])
└── 'x' --> <FastTreeValue 0x7fabb03816d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

29.5 µs ± 771 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fabb039df10>
├── 'a' --> tensor([[ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904],
│                   [ 0.9432,  0.5049, -0.6372],
│                   [ 0.2895,  0.3341, -0.0904]])
└── 'x' --> <FastTreeValue 0x7fac3c5c3fd0>
    └── 'c' --> tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                        [ 0.2619, -0.3360, -0.7763, -0.8405],
                 

In [23]:
%timeit t_cat(trees)

27.3 µs ± 907 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

52.7 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7929, -0.2308,  0.6232, -1.4665],
                       [ 0.2619, -0.3360, -0.7763, -0.8405],
                       [ 1.1931, -0.9129,  0.8167,  0.7423]],
              
                      [[-0.7929, -0.2308,  0.6232, -1.4665],
                       [ 0.2619, -0.3360, -0.7763, -0.8405],
                       [ 1.1931, -0.9129,  0.8167,  0.7423]],
              
                      [[-0.7929, -0.2308,  0.6232, -1.4665],
                       [ 0.2619, -0.3360, -0.7763, -0.8405],
                       [ 1.1931, -0.9129,  0.8167,  0.7423]],
              
                      [[-0.7929, -0.2308,  0.6232, -1.4665],
                       [ 0.2619, -0.3360, -0.7763, -0.8405],
                       [ 1.1931, -0.9129,  0.8167,  0.7423]],
              
                      [[-0.7929, -0.2308,  0.6232, -1.4665],
                       [ 0.2619, -0.3360, -0.7763, -0.8405],
                       [ 1.1931, -0.9129,  0.8167,  0.7423]],

In [26]:
%timeit Batch.stack(batches)

71.4 µs ± 900 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423],
                      [-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423],
                      [-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423],
                      [-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423],
                      [-0.7929, -0.2308,  0.6232, -1.4665],
                      [ 0.2619, -0.3360, -0.7763, -0.8405],
                      [ 1.1931, -0.9129,  0.8167,  0.7423],
                      [-0.7929, -0.2308,  0.6232, -1.4665],
                   

In [28]:
%timeit Batch.cat(batches)

128 µs ± 982 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

312 µs ± 3.89 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
